In [1]:
import numpy as np
import matplotlib.pyplot as plt

# --- Dependencias de lenstronomy y astropy ---
from astropy import units as u
from astropy.constants import G, c
from astropy.cosmology import FlatLambdaCDM

from lenstronomy.Util.param_util import phi_q2_ellipticity, shear_polar2cartesian
from lenstronomy.Data.pixel_grid import PixelGrid
from lenstronomy.Data.psf import PSF
from lenstronomy.LightModel.light_model import LightModel
from lenstronomy.LensModel.lens_model import LensModel
from lenstronomy.PointSource.point_source import PointSource
from lenstronomy.ImSim.image_model import ImageModel
import lenstronomy.Util.image_util as image_util

In this block we:

1. **Define the lens and source redshifts** (`z_lens` and `z_source`) to match the system studied in Vegetti et al. (JVAS B1938+666 at $z=0.881$ and background source at $z=2.059$).  
2. **Instantiate a flat $\Lambda$CDM cosmology** with $H_0 = 70\;\mathrm{km\,s^{-1}\,Mpc^{-1}}$, $\Omega_m = 0.3$, and $\Omega_b = 0.048$.  
3. **Compute the angular diameter distances**:
   - $D_d$: from observer to lens  
   - $D_s$: from observer to source  
   - $D_{ds}$: from lens to source  

These distances are crucial for converting between physical masses and angular scales—most importantly to calculate the Einstein radius of the subhalo via  

$$
\theta_E = \sqrt{\frac{4G\,M}{c^2}\,\frac{D_{ds}}{D_d\,D_s}}
$$

and for any further lensing–cosmology conversions in the simulation.


In [2]:
# --- Parámetros cosmológicos y distancias ---
z_lens, z_source = 0.881, 2.059
cosmo = FlatLambdaCDM(H0=70, Om0=0.3, Ob0=0.048)
D_d  = cosmo.angular_diameter_distance(z_lens)
D_s  = cosmo.angular_diameter_distance(z_source)
D_ds = cosmo.angular_diameter_distance_z1z2(z_lens, z_source)


In this block we:

1. **Set the angular pixel scale** (`deltaPix = 0.05″/pixel`), defining how many arcseconds each pixel spans.  
2. **Choose the sky coordinates of the top-left corner** (`ra_at_xy_0 = –2.5″`, `dec_at_xy_0 = –2.5″`), so that a 100×100 grid covers a 5″×5″ field centered at (0,0).  
3. **Build the WCS linear transformation** matrix  
   $$
     \begin{pmatrix}
       \Delta\text{RA}_x & 0\\
       0 & \Delta\text{DEC}_y
     \end{pmatrix}
     = \Delta_{\rm pix}\times I
   $$
   which maps integer pixel shifts into sky‐angle shifts.  
4. **Instantiate `PixelGrid`** with these parameters, creating two 100×100 arrays (`x_coords`, `y_coords`) that give the RA/DEC of every pixel center.  

This object is the foundation for all ray‐tracing and surface‐brightness evaluations, ensuring that every model component uses the same angular coordinate system.  
